In [63]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np

In [64]:
train_set, train_lengths, val_set, _ = load_datasets()

In [65]:
def load_song(index):
    start_pos = train_lengths.iloc[:index].sum().item()
    return train_set.iloc[start_pos : start_pos + train_lengths.iloc[index].item()]

In [66]:
def load_songs(indices):
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs
    return pd.concat(songs)

song_indices = list(range(10))
songs = load_songs(song_indices)

In [80]:
states, observations = dataframe_to_states(
    songs, 
    2, 
    1
)

Processing states: 100%|██████████| 944/944 [00:00<00:00, 8155.84it/s]


In [81]:
transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(states, observations)

In [82]:
observation_indices = observations.astype(int) 

In [83]:
train_lengths.min()

lengths    7
dtype: int64

In [84]:
model = hmm.CategoricalHMM(
    n_components=transition_matrix.shape[0]
)

model.fit(observation_indices.reshape(-1, 1))

Fitting a model with 55289 free scalar parameters with only 944 data points will result in a degenerate solution.


CategoricalHMM(n_components=194, n_features=np.int64(92),
               random_state=RandomState(MT19937) at 0x7F69DD14E540)

In [85]:
model.transmat = transition_matrix.T
model.emissionprob = emission_probs.T
start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]
model.startprob = start_probs

Other tools:
- dynamax
  - runs on JAX
  - might have a more flexible structure
  - 

In [86]:
likelihood, pred_states = model.decode(observation_indices.reshape(-1, 1))
pred_states

array([ 67, 191, 182, 184, 173,  20, 180, 182,  78, 105, 160,  10, 136,
       130,   4,  28, 148,  99,  93,  16, 174,  19,  52, 173,  20, 180,
       182,  78, 105, 160, 109,  15,   4,  28, 148,  99,  93,  16,   8,
        16,   8,  16, 174,  19,  52, 101, 143, 164,  30, 109,  15,   4,
        63,  61, 178,  69,   8,  16,   8,  16, 174,  19,  52, 101, 109,
        15,   4,  28, 148,  99,  93,  55,  32,  53,  28, 148,  99,  93,
        42, 121,  16,   8,  16,   8,  16,   8,  16, 174,  19,  52,  41,
        82,  28, 148,  99,  93,  42, 128, 173,  20, 180, 182, 184, 173,
        20, 180,  56, 157,   3,  25, 162,  72,  73, 131, 163,  14, 133,
        13,   5,  33, 149,  44,   2, 165, 130, 157,   3,  25,  57,  92,
       127,  36,   6, 104,  29, 132, 133, 188, 146,  51, 150, 154,  25,
        57,  92, 127,  11,  57,  92, 127, 182, 184, 173,  98,  27,  36,
         6, 104,  29,  22, 137,  54, 136, 130, 104,  29,  22,  88,  78,
        12, 138, 191, 182, 184, 173,  98,  27,  36,   6, 184, 17